In [1]:
import pandas as pd
import numpy as np
from sklearn.naive_bayes import MultinomialNB
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from sklearn.multiclass import OneVsRestClassifier
import string
import ast
import re
import warnings
from meter import *
warnings.filterwarnings('ignore')

In [2]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Input, GlobalMaxPooling1D, Dropout
from keras.layers import Conv1D, MaxPooling1D, Embedding, Flatten, LSTM
from keras.models import Model, Sequential, load_model
from keras.optimizers import Adam

Using TensorFlow backend.


### GPU data preprocessing

In [4]:
# from gensim.models.word2vec import Word2Vec

In [7]:
# MAX_SEQUENCE_LENGTH = 200
# EMBEDDING_DIM = 300

# w2v_model = Word2Vec([t.split() for t in train_texts], size=EMBEDDING_DIM, min_count=1)

# tokenizer = Tokenizer(num_words=None)
# tokenizer.fit_on_texts(train_texts)
# word_index = tokenizer.word_index
# num_words = len(word_index) + 1

# sequences = tokenizer.texts_to_sequences(train_texts)
# X_w2v_train = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

# test_sequences = tokenizer.texts_to_sequences(test_texts)
# X_w2v_test = pad_sequences(test_sequences, 
#                              maxlen=MAX_SEQUENCE_LENGTH)

In [8]:
# w2v_matrix = np.zeros((num_words, EMBEDDING_DIM))

# for word, i in word_index.items():
# #     if i >= num_words:
# #         continue
#     if word in w2v_model.wv:
#         embedding_vector = w2v_model.wv[word]
#     else:
#         embedding_vector = 0
#     if embedding_vector is not None:
#         # words not found in embedding index will be all-zeros.
#         w2v_matrix[i] = embedding_vector

In [101]:
# np.save('w2v_matrix.npy', w2v_matrix)
# np.save('X_w2v_train.npy', X_w2v_train)
# np.save('y_train.npy', y_train)
# np.save('X_w2v_test.npy', X_w2v_test)
# np.save('y_test.npy', y_test)

### Load data

In [3]:
w2v_matrix = np.load('w2v_matrix.npy')
X_w2v_train = np.load('X_w2v_train.npy')
y_train = np.load('y_train.npy')
X_w2v_test = np.load('X_w2v_test.npy')
y_test = np.load('y_test.npy')

In [12]:
# MAX_SEQUENCE_LENGTH = 200
# EMBEDDING_DIM = 300

In [19]:
# lstm_w2v = Sequential()
# e = Embedding(w2v_matrix.shape[0], EMBEDDING_DIM, 
#               weights=[w2v_matrix], 
#               input_length=MAX_SEQUENCE_LENGTH, 
#               trainable=False)
# lstm_w2v.add(e)
# # lstm_w2v.add(Dropout(0.5))
# # lstm_w2v.add(Conv1D(40, 2))
# lstm_w2v.add(LSTM(80, dropout = 0.5, recurrent_dropout=0.5))
# # lstm_w2v.add(Dense(40, activation='relu'))
# lstm_w2v.add(Dense(20, activation='sigmoid'))
# lstm_w2v.compile(optimizer=Adam(lr=0.002), loss='binary_crossentropy', metrics=['acc'])

In [23]:
# lstm_w2v.compile(optimizer=Adam(lr=0.001), loss='binary_crossentropy', metrics=['acc'])
# lstm_w2v.fit(X_w2v_train, y_train, validation_data= (X_w2v_test, y_test),
#                batch_size=128,
#                epochs=100)

Train on 3489 samples, validate on 873 samples
Epoch 1/100
3489/3489 [==============================] - 10s 3ms/step - loss: 0.3242 - acc: 0.8661 - val_loss: 0.3387 - val_acc: 0.8595
Epoch 2/100
3489/3489 [==============================] - 9s 3ms/step - loss: 0.3240 - acc: 0.8655 - val_loss: 0.3385 - val_acc: 0.8583
Epoch 3/100
3489/3489 [==============================] - 9s 3ms/step - loss: 0.3229 - acc: 0.8665 - val_loss: 0.3379 - val_acc: 0.8588
Epoch 4/100
3489/3489 [==============================] - 9s 3ms/step - loss: 0.3236 - acc: 0.8660 - val_loss: 0.3391 - val_acc: 0.8590
Epoch 5/100
3489/3489 [==============================] - 9s 3ms/step - loss: 0.3253 - acc: 0.8645 - val_loss: 0.3391 - val_acc: 0.8580
Epoch 6/100
3489/3489 [==============================] - 9s 3ms/step - loss: 0.3225 - acc: 0.8651 - val_loss: 0.3390 - val_acc: 0.8576
Epoch 7/100
3489/3489 [==============================] - 9s 3ms/step - loss: 0.3228 - acc: 0.8652 - val_loss: 0.3396 - val_acc: 0.8579
Epoch 8

Epoch 61/100
3489/3489 [==============================] - 9s 3ms/step - loss: 0.3110 - acc: 0.8692 - val_loss: 0.3407 - val_acc: 0.8580
Epoch 62/100
3489/3489 [==============================] - 9s 3ms/step - loss: 0.3110 - acc: 0.8699 - val_loss: 0.3402 - val_acc: 0.8581
Epoch 63/100
3489/3489 [==============================] - 9s 3ms/step - loss: 0.3092 - acc: 0.8713 - val_loss: 0.3421 - val_acc: 0.8586
Epoch 64/100
3489/3489 [==============================] - 9s 3ms/step - loss: 0.3099 - acc: 0.8705 - val_loss: 0.3406 - val_acc: 0.8594
Epoch 65/100
3489/3489 [==============================] - 9s 3ms/step - loss: 0.3106 - acc: 0.8690 - val_loss: 0.3406 - val_acc: 0.8585
Epoch 66/100
3489/3489 [==============================] - 9s 3ms/step - loss: 0.3092 - acc: 0.8712 - val_loss: 0.3404 - val_acc: 0.8584
Epoch 67/100
3489/3489 [==============================] - 9s 3ms/step - loss: 0.3086 - acc: 0.8697 - val_loss: 0.3415 - val_acc: 0.8573
Epoch 68/100
3489/3489 [========================

In [44]:
# lstm_w2v.save('lstm_w2v_55_25_150')

In [4]:
lstm_w2v = load_model('lstm_w2v_55_25_150')

In [5]:
lstm_w2v.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 200, 300)          13495200  
_________________________________________________________________
lstm_4 (LSTM)                (None, 80)                121920    
_________________________________________________________________
dense_4 (Dense)              (None, 20)                1620      
Total params: 13,618,740
Trainable params: 123,540
Non-trainable params: 13,495,200
_________________________________________________________________


In [6]:
lstm_pred_w2v_test = lstm_w2v.predict(X_w2v_test)
# evaluation
print('Word2Vec Avg Accuracy on test : {}'.format(score_thres(y_test, lstm_pred_w2v_test, method='avg')))
print('Word2Vec Exact Accuracy on test : {}'.format(score_thres(y_test, lstm_pred_w2v_test, method='exact')))
print('Word2Vec Precision on test : {}'.format(score_thres(y_test, lstm_pred_w2v_test, method='precision')))
print('Word2Vec Recall on test : {}'.format(score_thres(y_test, lstm_pred_w2v_test, method='recall')))
print('Word2Vec Hit Rate on test : {}'.format(score_thres(y_test, lstm_pred_w2v_test, method='hit')))
print('Word2VecF1 on test : {}'.format(score_thres(y_test, lstm_pred_w2v_test, method='f1')))


Word2Vec Avg Accuracy on test : 0.8624856815578466
Word2Vec Exact Accuracy on test : 0.061855670103092786
Word2Vec Precision on test : 0.5525787967604396
Word2Vec Recall on test : 0.2585768591891645
Word2Vec Hit Rate on test : 0.7800687285223368
Word2VecF1 on test : 0.3522975859758274


#### Another model...

In [7]:
lstm_w2v = load_model('lstm_w2v_45_27_120')

In [8]:
lstm_w2v.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 200, 300)          13495200  
_________________________________________________________________
lstm_3 (LSTM)                (None, 50)                70200     
_________________________________________________________________
dense_3 (Dense)              (None, 20)                1020      
Total params: 13,566,420
Trainable params: 71,220
Non-trainable params: 13,495,200
_________________________________________________________________


In [9]:
lstm_pred_w2v_test = lstm_w2v.predict(X_w2v_test)
# evaluation
print('Word2Vec Avg Accuracy on test : {}'.format(score_thres(y_test, lstm_pred_w2v_test, method='avg')))
print('Word2Vec Exact Accuracy on test : {}'.format(score_thres(y_test, lstm_pred_w2v_test, method='exact')))
print('Word2Vec Precision on test : {}'.format(score_thres(y_test, lstm_pred_w2v_test, method='precision')))
print('Word2Vec Recall on test : {}'.format(score_thres(y_test, lstm_pred_w2v_test, method='recall')))
print('Word2Vec Hit Rate on test : {}'.format(score_thres(y_test, lstm_pred_w2v_test, method='hit')))
print('Word2VecF1 on test : {}'.format(score_thres(y_test, lstm_pred_w2v_test, method='f1')))


Word2Vec Avg Accuracy on test : 0.8585337915234822
Word2Vec Exact Accuracy on test : 0.06529209621993128
Word2Vec Precision on test : 0.4576114086200838
Word2Vec Recall on test : 0.2703722831824408
Word2Vec Hit Rate on test : 0.7835051546391752
Word2VecF1 on test : 0.33991267318803375
